In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split

In [3]:
sim = np.load("./hm_feedback_sim.npy")
bio = np.load("./hm_feedback_bio.npy")

sim.shape, bio.shape

((48, 4), (80, 4))

# train fusion LR

In [50]:
X_train_mlp = np.load("collaboration_with_ting_zhang/result/preds_sim_train_0.2.npy")
X_train_rf = np.load("collaboration_with_ting_zhang/result/rf_pred_train_0.2.npy")
y_train = np.load("collaboration_with_ting_zhang/result/sim_training_label_0.2.npy")

X_test_mlp = np.load("collaboration_with_ting_zhang/result/preds_sim_test_0.8.npy")
X_test_rf = np.load("collaboration_with_ting_zhang/result/rf_pred_test_0.8.npy")
y_test = np.load("collaboration_with_ting_zhang/result/sim_testing_label_0.8.npy")

X_train = np.hstack([X_train_mlp.reshape(-1,1), X_train_rf.reshape(-1,1)])
X_test = np.hstack([X_test_mlp.reshape(-1,1), X_test_rf.reshape(-1,1)])

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((460, 2), (460, 1), (1843, 2), (1843, 1))

In [84]:
print("train - mlp only:", accuracy_score(y_true=y_train, y_pred=X_train_mlp.round()))
print("train - rf only: ", accuracy_score(y_true=y_train, y_pred=X_train_rf.round()))
print("test - mlp only:", accuracy_score(y_true=y_test, y_pred=X_test_mlp.round()))
print("test - rf only: ", accuracy_score(y_true=y_test, y_pred=X_test_rf.round()))

train - mlp only: 1.0
train - rf only:  0.9934782608695653
test - mlp only: 0.8643516006511123
test - rf only:  0.951709169831796


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV

fusion = LogisticRegression()
# fusion = RandomForestClassifier()
# fusion = SVC(kernel='rbf')
# fusion = GaussianProcessClassifier()
# fusion = LGBMClassifier()

fusion.fit(X_train, y_train.reshape(-1))

train_acc = accuracy_score(y_true=y_train, y_pred=fusion.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=fusion.predict(X_test))

print("train acc: ", train_acc)
print("test acc", test_acc)

train - mlp only: 1.0
train - rf only:  0.9934782608695653
test - mlp only: 0.8643516006511123
test - rf only:  0.951709169831796


In [ ]:
# fusion_test_pred = fusion.predict(X_test)

# for i in range(len(X_test)):
#     if fusion_test_pred[i] != sim_test_label[i]:
#         print(sim_test_data[i], sim_test_label[i])

In [79]:
import torch

torch.save(torch.from_numpy(X_train), "collaboration_with_ting_zhang/result/fusion_train_X.pth")
torch.save(torch.from_numpy(y_train), "collaboration_with_ting_zhang/result/fusion_train_y.pth")
torch.save(torch.from_numpy(X_test), "collaboration_with_ting_zhang/result/fusion_test_X.pth")
torch.save(torch.from_numpy(y_test), "collaboration_with_ting_zhang/result/fusion_test_y.pth")

In [80]:
error_train_X = torch.hstack([
    torch.from_numpy(sim_train_data), 
    torch.from_numpy(X_train), 
    torch.from_numpy(fusion.predict_proba(X_train)[:,1]).view(-1,1)])
error_train_y = torch.from_numpy(y_train) - torch.from_numpy(fusion.predict_proba(X_train)[:,1]).view(-1,1)

error_test_X = torch.hstack([
    torch.from_numpy(sim_test_data), 
    torch.from_numpy(X_test), 
    torch.from_numpy(fusion.predict_proba(X_test)[:,1]).view(-1,1)])
error_test_y = torch.from_numpy(y_test) - torch.from_numpy(fusion.predict_proba(X_test)[:,1]).view(-1,1)

print("error train: ", error_train_X.shape, error_train_y.shape)
print("error test: ", error_test_X.shape, error_test_y.shape)

torch.save(error_train_X, "collaboration_with_ting_zhang/result/error_train_X.pth")
torch.save(error_train_y, "collaboration_with_ting_zhang/result/error_train_y.pth")
torch.save(error_test_X, "collaboration_with_ting_zhang/result/error_test_X.pth")
torch.save(error_test_y, "collaboration_with_ting_zhang/result/error_test_y.pth")

error train:  torch.Size([460, 9]) torch.Size([460, 1])
error test:  torch.Size([1843, 9]) torch.Size([1843, 1])


In [83]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor

# error_corr = LGBMRegressor()
error_corr = GaussianProcessRegressor()

error_corr.fit(error_train_X, error_train_y.reshape(-1))

train_mse = mean_squared_error(y_true=error_train_y, y_pred=error_corr.predict(error_train_X))
test_mse = mean_squared_error(y_true=error_test_y, y_pred=error_corr.predict(error_test_X))

print("train mse: ", train_mse)
print("test mse", train_mse)

new_pred_train = error_corr.predict(error_train_X) + error_train_X[:, -1].numpy().reshape(-1)
new_pred_test = error_corr.predict(error_test_X) + error_test_X[:, -1].numpy().reshape(-1)

train_acc = accuracy_score(y_true=y_train.reshape(-1), y_pred=new_pred_train.round())
test_acc = accuracy_score(y_true=y_test.reshape(-1), y_pred=new_pred_test.round())

print("train acc: ", train_acc)
print("test acc: ", test_acc)

train mse:  7.16483782871675e-24
test mse 7.16483782871675e-24
train acc:  1.0
test acc:  0.893651654910472


# train RF 

##### on [original + cross feats] with [2:8 train-test]

In [21]:
sim_train_data = np.load("collaboration_with_ting_zhang/result/sim_training_data_0.2.npy")
sim_train_label = np.load("collaboration_with_ting_zhang/result/sim_training_label_0.2.npy")

sim_test_data = np.load("collaboration_with_ting_zhang/result/sim_testing_data_0.8.npy")
sim_test_label = np.load("collaboration_with_ting_zhang/result/sim_testing_label_0.8.npy")

print("train: ", sim_train_data.shape, sim_train_label.shape)
print("test: ", sim_test_data.shape, sim_test_label.shape)

train:  (460, 6) (460, 1)
test:  (1843, 6) (1843, 1)


In [22]:
sim_training_data_maxmin = np.load("collaboration_with_ting_zhang/result/sim_training_data_maxmin0.2.npy")
sim_training_label_maxmin = np.load("collaboration_with_ting_zhang/result/sim_training_label_maxmin0.2.npy")

# sim_testing_data_maxmin = np.load("collaboration_with_ting_zhang/result/sim_testing_data_maxmin0.8.npy")
# sim_testing_label_maxmin = np.load("collaboration_with_ting_zhang/result/sim_testing_data_maxmin0.8.npy")

sim_train_data = sim_train_data * (sim_training_data_maxmin[0, :] - sim_training_data_maxmin[1, :]) \
    + sim_training_data_maxmin[1, :]
sim_train_label = sim_train_label * (sim_training_label_maxmin[0, :] - sim_training_label_maxmin[1, :]) \
    + sim_training_label_maxmin[1, :]

sim_test_data = sim_test_data * (sim_training_data_maxmin[0, :] - sim_training_data_maxmin[1, :]) \
    + sim_training_data_maxmin[1, :]
sim_test_label = sim_test_label * (sim_training_label_maxmin[0, :] - sim_training_label_maxmin[1, :]) \
    + sim_training_label_maxmin[1, :]

print("train: ", sim_train_data.shape, sim_train_label.shape)
print("test: ", sim_test_data.shape, sim_test_label.shape)

train:  (460, 6) (460, 1)
test:  (1843, 6) (1843, 1)


In [46]:
sim_train_data

array([[50. ,  1. ,  2. , 10. ,  0.5,  5. ],
       [20. , 10. ,  5. , 50. ,  5. ,  2. ],
       [50. ,  5. ,  5. , 50. ,  1. ,  5. ],
       ...,
       [10. ,  0.5,  2. , 50. , 10. ,  2. ],
       [15. ,  5. ,  5. , 10. ,  1. ,  5. ],
       [15. ,  1. ,  5. , 15. , 10. ,  5. ]])

In [6]:
cnt = 0
for i in range(len(sim_train_data)):
    for j in range(len(sim_test_data)):
        if np.array_equal(sim_train_data[i], sim_test_data[j]):
            print(i, j)
            cnt += 1

print(cnt)

0 473
1 134
2 493
4 821
5 1648
6 1512
7 162
8 1004
10 1089
11 67
12 742
14 1649
15 1221
16 483
17 508
18 1225
19 93
20 286
21 1187
22 573
24 1546
25 715
26 1091
29 443
31 669
33 1122
34 701
35 1662
36 75
38 800
39 1341
40 776
41 975
42 183
44 49
46 5
47 752
48 723
49 1045
50 1784
51 1430
52 572
53 1760
54 211
55 476
56 397
59 104
60 100
61 101
62 1541
63 1391
64 23
65 1435
66 592
67 1305
68 1717
69 1582
70 259
71 647
74 1773
75 470
78 999
79 1293
80 1801
81 297
82 642
83 1000
84 1810
85 1674
88 718
89 1499
90 535
91 873
92 1474
94 635
95 400
97 1584
99 369
100 845
102 1304
103 1699
104 1484
105 1679
106 1422
107 423
108 1048
109 1593
110 726
111 546
112 1188
113 311
114 1554
115 313
116 322
117 1216
118 928
119 1458
121 1105
122 833
123 1148
124 1319
126 1059
127 299
129 438
130 414
131 1770
132 537
133 453
134 695
135 739
136 232
137 1805
138 1761
139 935
140 1040
141 1364
142 329
143 1147
144 908
145 169
146 959
147 654
148 809
149 187
150 1466
151 39
152 167
153 1743
154 155
155 128

In [23]:
train = []
for i in range(len(sim_train_data)):
    feat = calc_theta_features(x1=sim_train_data[i, :3], x2=sim_train_data[i, 3:], y=sim_train_label.reshape(-1)[i], mode=1)
    train.append(np.array(feat))

test = []
for i in range(len(sim_test_data)):
    feat = calc_theta_features(x1=sim_test_data[i, :3], x2=sim_test_data[i, 3:], y=sim_test_label.reshape(-1)[i], mode=1)
    test.append(np.array(feat))

train = np.array(train)
test = np.array(test)

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

print("train: ", X_train.shape, y_train.shape)
print("test: ", X_test.shape, y_test.shape)

train:  (460, 20) (460,)
test:  (1843, 20) (1843,)


In [24]:
best = 0.0
best_param = []

ne_list=[50,100,200]
md_list=[8,12,16,32,None]
mss_list=[2,5,10,20]

for ne in ne_list:
    for md in md_list:
        for mss in mss_list:
                
            rf = RandomForestClassifier(n_estimators=ne, max_depth=md, min_samples_split=mss)

            rf.fit(X_train, y_train.reshape(-1))
            train_acc = accuracy_score(y_true=y_train, y_pred=rf.predict(X_train))
            test_acc = accuracy_score(y_true=y_test, y_pred=rf.predict(X_test))
            
            if test_acc > best:
                best = test_acc
                best_param = [ne, md, mss]
                print("best param: ", ne, md, mss)
                print("best train acc: ", train_acc)
                print("best test acc: ", test_acc)
                print("save to collaboration_with_ting_zhang/result/rf.pkl\n")
                pickle.dump(rf, open("collaboration_with_ting_zhang/result/rf.pkl", "wb"))
                np.save("collaboration_with_ting_zhang/result/rf_pred_train_0.2.npy", rf.predict_proba(X_train)[:, 1])
                np.save("collaboration_with_ting_zhang/result/rf_pred_test_0.8.npy", rf.predict_proba(X_test)[:, 1])

best param:  50 8 2
best train acc:  0.9956521739130435
best test acc:  0.9413998914812806
save to collaboration_with_ting_zhang/result/rf.pkl

best param:  50 8 5
best train acc:  0.991304347826087
best test acc:  0.9419424850786761
save to collaboration_with_ting_zhang/result/rf.pkl

best param:  50 16 2
best train acc:  1.0
best test acc:  0.9506239826370049
save to collaboration_with_ting_zhang/result/rf.pkl

best param:  50 None 2
best train acc:  1.0
best test acc:  0.9511665762344005
save to collaboration_with_ting_zhang/result/rf.pkl

best param:  50 None 5
best train acc:  0.9934782608695653
best test acc:  0.951709169831796
save to collaboration_with_ting_zhang/result/rf.pkl



# prepare data

In [9]:
# create pair (x1, x2, y)
# x1 & x2 are parameters, y=1 if x1 is preferred over x2

def calc_theta_features(x1, x2, y, mode=0):

    # only original feats
    if mode==0:
        feats = [x1[0], x1[1], x1[2], x2[0], x2[1], x2[2], y]

    # original feats + cross feats
    elif mode==1:
        feats = [
            x1[0], x1[1], x1[2], 
            x2[0], x2[1], x2[2], 
            x1[0]-x2[0], 
            x1[1]-x2[1], 
            x1[2]-x2[2], 
            x1[0]/x2[0], 
            x1[1]/x2[1], 
            x1[2]/x2[2], 
            x1[0]+x1[0]*x1[1]-x1[0]*x1[2], 
            x2[0]+x2[0]*x2[1]-x2[0]*x2[2], 
            (x1[0]+x1[0]*x1[1])/x1[0]*x1[2], 
            (x2[0]+x2[0]*x2[1])/x2[0]*x2[2], 
            x1[0]*x1[1], 
            x2[0]*x2[1], 
            x1[0]*x1[2], 
            x2[0]*x2[1], 
            y
        ]

    # only cross feats
    elif mode==2:
        feats = [
            # x1[0], x1[1], x1[2], 
            # x2[0], x2[1], x2[2], 
            x1[0]-x2[0], 
            x1[1]-x2[1], 
            x1[2]-x2[2], 
            x1[0]/x2[0], 
            x1[1]/x2[1], 
            x1[2]/x2[2], 
            x1[0]+x1[0]*x1[1]-x1[0]*x1[2], 
            x2[0]+x2[0]*x2[1]-x2[0]*x2[2], 
            (x1[0]+x1[0]*x1[1])/x1[0]*x1[2], 
            (x2[0]+x2[0]*x2[1])/x2[0]*x2[2], 
            x1[0]*x1[1], 
            x2[0]*x2[1], 
            x1[0]*x1[2], 
            x2[0]*x2[1], 
            y
        ]

    else:
        print("wrong mode!")

    return feats

def create_pref(data, mode=0):
    pairs = []
    for i in range(len(data)):
        for j in range(len(data)):
            y = 1 if data[i][-1] > data[j][-1] else 0
            pairs.append(calc_theta_features(data[i][:3], data[j][:3], y, mode=mode))
    pairs = np.array(pairs)
    print(pairs.shape)
    return pairs

In [7]:
sim_data0 = create_pref(sim, mode=0)
bio_data0 = create_pref(bio, mode=0)
sim_data1 = create_pref(sim, mode=1)
bio_data1 = create_pref(bio, mode=1)
sim_data2 = create_pref(sim, mode=2)
bio_data2 = create_pref(bio, mode=2)

NameError: name 'sim' is not defined

# Tree

In [84]:
def cross_val_rf(ne_list, md_list, mss_list, X, y, cv=5):

    best = [0.0, 0.0]
    best_param = []

    for ne in ne_list:
        for md in md_list:
            for mss in mss_list:
                    
                    rf = RandomForestClassifier(n_estimators=ne, max_depth=md, min_samples_split=mss)

                    scores = cross_val_score(rf, X, y, cv=cv)
                    if scores.mean() > best[0]:
                        best = [scores.mean(), scores.std()]
                        best_param = [ne, md, mss]
                        print("best param: ", ne, md, mss)
                        print("best acc: ", best[0], best[1])

In [41]:
X, y = sim_data0[:, :-1], sim_data0[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.9218956898990852 0.02865741467929385
best param:  50 8 5
best acc:  0.9236310478166556 0.025175969011542627
best param:  50 8 10
best acc:  0.9249306799962275 0.030736953884597025
best param:  50 12 2
best acc:  0.925804017730831 0.033036644254497524
best param:  50 12 5
best acc:  0.9292728473073659 0.0322930152890416
best param:  50 16 2
best acc:  0.9323097236631142 0.02480482307886147
best param:  100 32 2
best acc:  0.9336112421012921 0.023845984313836176


In [42]:
X, y = sim_data1[:, :-1], sim_data1[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.9366518909742526 0.03668526283071027
best param:  50 12 2
best acc:  0.9500971423182119 0.02711803247544511
best param:  100 32 2
best acc:  0.9514014901442988 0.022389563171431728


In [43]:
X, y = sim_data2[:, :-1], sim_data2[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.9288371215693673 0.036986646263492455
best param:  50 8 5
best acc:  0.9297048005281525 0.038119665014149526
best param:  50 12 2
best acc:  0.9388163727247004 0.028414689358960674


In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    sim_data2[:, :-1], sim_data2[:, -1], test_size=0.8, random_state=114514)

rf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=1)

rf.fit(X_train, y_train.reshape(-1))
print("train acc: ", accuracy_score(y_true=y_train, y_pred=rf.predict(X_train)))
print("test acc: ", accuracy_score(y_true=y_test, y_pred=rf.predict(X_test)))

train acc:  1.0
test acc:  0.952819956616052


In [11]:
X, y = bio_data0[:, :-1], bio_data0[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.72046875 0.014785246552729525
best param:  50 8 20
best acc:  0.72265625 0.026773062488254885
best param:  100 8 10
best acc:  0.7254687500000001 0.021950761661044944
best param:  200 8 10
best acc:  0.72609375 0.023147159990482603
best param:  200 8 20
best acc:  0.7284375 0.025129547163150388


In [12]:
X, y = bio_data1[:, :-1], bio_data1[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.714375 0.06656249999999998
best param:  50 8 5
best acc:  0.71671875 0.07061670305246488
best param:  50 8 20
best acc:  0.7215625 0.06061332039556155
best param:  100 8 10
best acc:  0.7225 0.06354094091115585


In [11]:
X, y = bio_data2[:, :-1], bio_data2[:, -1]

cross_val_rf(ne_list=[50,100,200], md_list=[8,12,16,32,None], mss_list=[2,5,10,20], X=X, y=y, cv=5)

best param:  50 8 2
best acc:  0.7120312500000001 0.061292236712735496
best param:  50 8 5
best acc:  0.724375 0.06192982104265277
best param:  50 8 20
best acc:  0.7259375 0.05306384347533636


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    bio_data2[:, :-1], bio_data2[:, -1], test_size=0.8, random_state=114514)

rf = RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_split=10)

rf.fit(X_train, y_train.reshape(-1))
print("train acc: ", accuracy_score(y_true=y_train, y_pred=rf.predict(X_train)))
print("test acc: ", accuracy_score(y_true=y_test, y_pred=rf.predict(X_test)))

train acc:  0.8875
test acc:  0.8123046875


In [18]:
scores = cross_val_score(rf, X=bio_data2[:, :-1], y=bio_data2[:, -1], cv=5)
scores, scores.mean(), scores.std()

(array([0.746875  , 0.7390625 , 0.78203125, 0.59375   , 0.703125  ]),
 0.7129687499999999,
 0.06466862647373917)

# GP

In [50]:
def cross_val_gp(kernel_list, X, y, cv=5):

    best = [0.0, 0.0]
    best_param = []

    for kernel in kernel_list:
                
        gpc = GaussianProcessClassifier(kernel=kernel).fit(X, y)

        scores = cross_val_score(gpc, X, y, cv=cv)
        if scores.mean() > best[0]:
            best = [scores.mean(), scores.std()]
            print("best kernel: ", kernel)
            print("best acc: ", best[0], best[1])

In [53]:
kernel_list = [
    1.0 * RBF(1.0), 
    1.0 * Matern(1.0), 
    1.0 * RationalQuadratic(1.0), 
    1.0 * ExpSineSquared(1.0), 
    1.0 * DotProduct(1.0)
]

In [54]:
X, y = sim_data0[:, :-1], sim_data0[:, -1]

cross_val_gp(kernel_list=kernel_list, X=X, y=y, cv=5)

best kernel:  1**2 * RBF(length_scale=1)
best acc:  0.6141497689333207 0.0005338111855135441


In [55]:
X, y = sim_data1[:, :-1], sim_data1[:, -1]

cross_val_gp(kernel_list=kernel_list, X=X, y=y, cv=5)

best kernel:  1**2 * RBF(length_scale=1)
best acc:  0.6141497689333207 0.0005338111855135441


In [56]:
X, y = bio_data0[:, :-1], bio_data0[:, -1]

cross_val_gp(kernel_list=kernel_list, X=X, y=y, cv=5)

best kernel:  1**2 * RBF(length_scale=1)
best acc:  0.6884375 0.0003125000000000267


In [ ]:
X, y = bio_data1[:, :-1], bio_data1[:, -1]

cross_val_gp(kernel_list=kernel_list, X=X, y=y, cv=5)